<a href="https://colab.research.google.com/github/alvisaung/dl-hw/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="markdown-google-sans">

## Machine learning
</div>

With Colab you can import an image dataset, train an image classifier on it, and evaluate the model, all in just [a few lines of code](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb).

Colab is used extensively in the machine learning community with applications including:
- Getting started with TensorFlow
- Developing and training neural networks
- Experimenting with TPUs
- Disseminating AI research
- Creating tutorials

To see sample Colab notebooks that demonstrate machine learning applications, see the [machine learning examples](#machine-learning-examples) below.

In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import drive

drive.mount('/content/drive')

with open('/content/drive/My Drive/fish/shakespeare_train.txt', 'r') as f:
    train_text = f.read()

with open('/content/drive/My Drive/fish/shakespeare_valid.txt', 'r') as f:
    valid_text = f.read()

# Create vocabulary and mappings
vocab = sorted(set(train_text))
vocab_size = len(vocab)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))



# Convert text to integers
train_data = np.array([vocab_to_int[c] for c in train_text], dtype=np.int32)
valid_data = np.array([vocab_to_int[c] for c in valid_text], dtype=np.int32)

# temp
temp_size = 20000
temp_size_v = 1000
train_data = train_data[:temp_size]  # Use the first 100,000 characters
valid_data = valid_data[:temp_size_v]
print(train_data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[18 49 58 ... 55  6  1]


In [2]:

def build_rnn_model(vocab_size):
    hidden_units = 512  # You can experiment with different sizes
    model = models.Sequential([
        layers.Embedding(input_dim=vocab_size, output_dim=hidden_units),
        layers.SimpleRNN(hidden_units, return_sequences=True),
        layers.Dense(vocab_size, activation='softmax')  # Output probabilities over characters
    ])
    return model

model = build_rnn_model(vocab_size)

In [3]:
import matplotlib.pyplot as plt

def plot_loss(history):
    plt.plot(history.history['loss'], label='Training BPC')
    plt.plot(history.history['val_loss'], label='Validation BPC')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Bits Per Character')
    plt.title('Learning Curve')
    plt.show()

In [4]:
def bpc_loss(y_true, y_pred):

    y_true = tf.cast(y_true, tf.int32)
    scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = scce(y_true, y_pred)
    loss_in_bits = loss / tf.math.log(2.0)
    return tf.reduce_mean(loss_in_bits)


In [5]:
def create_tf_dataset(data, seq_length, batch_size):
    # Convert data to TensorFlow Dataset
    dataset = tf.data.Dataset.from_tensor_slices(data)

    # Create sequences
    sequences = dataset.window(seq_length + 1, shift=1, drop_remainder=True)
    sequences = sequences.flat_map(lambda window: window.batch(seq_length + 1))

    # Split sequences into input and target
    def split_input_target(seq):
        input_seq = seq[:-1]
        target_seq = seq[1:]
        return input_seq, target_seq

    dataset = sequences.map(split_input_target)

    # Shuffle, batch, and prefetch
    dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

    return dataset

seq_length = 100
batch_size = 64

train_dataset = create_tf_dataset(train_data, seq_length, batch_size)
valid_dataset = create_tf_dataset(valid_data, seq_length, batch_size)


In [6]:
batch_size = 64
optimizer = tf.keras.optimizers.Adam()

# Define a custom callback to reset hidden states at the start of each epoch
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_epoch_{epoch}.weights.h5',
    save_weights_only=True,
    save_freq='epoch'
)

def generate_text(model, start_string, num_generate=100):
    # Convert start string to integer indices
    input_indices = [vocab_to_int[c] for c in start_string]
    input_indices = tf.expand_dims(input_indices, 0)  # Add batch dimension

    text_generated = []
    model.reset_states()

    for _ in range(num_generate):
        predictions = model(input_indices)
        predictions = tf.squeeze(predictions, 0)  # Remove batch dimension

        # Use the last prediction
        predictions = predictions[-1]

        # Sample from the distribution
        predicted_id = tf.random.categorical(predictions[None, :], num_samples=1)[0, 0].numpy()

        # Add predicted character
        text_generated.append(int_to_vocab[predicted_id])

        # Update input
        input_indices = tf.expand_dims([predicted_id], 0)

    return start_string + ''.join(text_generated)


def experiment(epochs):
    hidden_sizes = [128, 256, 512]
    sequence_lengths = [50, 100, 200]

    for hidden_units in hidden_sizes:
        for seq_length in sequence_lengths:
            # Build and compile the model
            model = build_rnn_model(vocab_size, hidden_units)
            model.compile(optimizer=optimizer, loss=bpc_loss)


            # Train the model
            history = model.fit(
                train_dataset,validation_data=valid_dataset, epochs=epochs,callbacks=[checkpoint_callback]
            )

            # Record the final training loss
            final_loss = history.history['loss'][-1]
            print(f'Hidden Units: {hidden_units}, Seq Length: {seq_length}, Final Loss: {final_loss}')


In [7]:
model.compile(optimizer=optimizer, loss=bpc_loss)
epochs = 2

# Fit the model
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    callbacks=[checkpoint_callback]
)



Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:609: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


    310/Unknown 209s 663ms/step - loss: 3.2143

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


310/310 ━━━━━━━━━━━━━━━━━━━━ 213s 675ms/step - loss: 3.2114 - val_loss: 4.5511
Epoch 2/2
310/310 ━━━━━━━━━━━━━━━━━━━━ 256s 654ms/step - loss: 0.6851 - val_loss: 5.5968


In [8]:
model.load_weights('model_epoch_5.weights.h5')  # For example, epoch 5

# Generate text
start_string = "JULIET"
generated_text = generate_text(model, start_string, num_generate=500)
print(generated_text)
# Load model weights from a specific epoch
model.load_weights('model_epoch_10.weights.h5')  # For example, epoch 5

# Generate text
start_string = "JULIET"
generated_text = generate_text(model, start_string, num_generate=500)
print(generated_text)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'model_epoch_5.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)